In [0]:
%run ../Utils/DBInstanceUtils

In [0]:
%run ../Utils/ProcessingUtils

In [0]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [0]:
dbi = DBInstanceUtils('treinamento_formula1')
filename = 'raceRaw'
extension = 'delta'
file = f'{filename}.{extension}'

DBInstanceUtils.set_dbfs_filepath(dbi, 'mnt',f'formula1/bronze/race', file)

In [0]:
df = spark.read.format(extension).load(f'{dbi.get_dbfs_filepath()}').drop('url')

df = (Refine(df)
            .camelCase_to_snake_case()
            .add_constcol('ingestion_date',T.TimestampType(), F.current_timestamp())
            .load())

In [0]:
filename = 'raceTrusted'
file = f'{filename}.{extension}'
DBInstanceUtils.set_dbfs_savepath(dbi,'mnt','formula1/prata/race',file)

In [0]:
df.write.format('delta').mode('overwrite').save(dbi.get_dbfs_savepath())